# Data preparation

In order to prepare the data to run the models and make predictions, we need to clean it and manipulate the attributes in order to improve it.

To do so, we have developed some functions that will read the datasets, clean them, and then join them making for the final train and test datasets.

In [131]:
if (!require("dplyr")) install.packages("dplyr")

## Account Data

On the account dataset, we decided to parse the date from the YYMMDD format to YYYY-MM-DD, and then use it to calculate a new attribute called "age_days", which stores the age of the account in days. This will be useful to be able to compare statistics from accounts with different ages or other derived attributes.

In [132]:
prepare_account <- function() {
  # Load dataset
  account_data <- read.csv("../data/account.csv", sep = ";")
  account_data <-
    replace(account_data, (account_data == "" | account_data == " "), NA)

  # Make date more readable
  account_data <- transform(account_data, acc_creation_date = as.Date(
    paste(
      paste("19", date %/% 10000, sep = ""),
      (date %/% 100) %% 100,
      date %% 100,
      sep = "-"
    ),
    format = "%Y-%m-%d"
  ))

  # Change string variables to integers, works better for later steps
  account_data[account_data$frequency == "monthly issuance", ]$frequency <- 1
  account_data[account_data$frequency == "weekly issuance", ]$frequency <- 2
  account_data[
    account_data$frequency == "issuance after transaction",
  ]$frequency <- 3

  # Calculate age of account in days
  account_data$age_days <- trunc(as.numeric(
    difftime(Sys.Date(), account_data$acc_creation_date, units = "days")
  ))

  return(account_data)
}

In [133]:
account <- prepare_account()
head(account)

,account_id,district_id,frequency,date,acc_creation_date,age_days
,<int>,<int>,<chr>,<int>,<date>,<dbl>
1,576,55,1,930101,1993-01-01,10921
2,3818,74,1,930101,1993-01-01,10921
3,704,55,1,930101,1993-01-01,10921
4,2378,16,1,930101,1993-01-01,10921
5,2632,24,1,930102,1993-01-02,10920
6,1972,77,1,930102,1993-01-02,10920


## Card data

For the card dataset, we only changed the data format and got rid of the day and month.

In [134]:
prepare_card <- function(train = TRUE) {
  card_data <- read.csv(if_else(train,
    "../data/card_dev.csv", "../data/card_comp.csv"),
  sep = ";")
  card_data <- replace(card_data, (card_data == "" | card_data == " "), NA)
  # Make date more readable
  card_data <- transform(card_data, issued = format(as.Date(
    paste(
      paste("19", issued %/% 10000, sep = ""),
      (issued %/% 100) %% 100,
      issued %% 100,
      sep = "-"
    ),
    format = "%Y-%m-%d"
  ), "%Y"))
  return(card_data)
}

In [135]:
card <- prepare_card()
head(card)

,card_id,disp_id,type,issued
,<int>,<int>,<chr>,<chr>
1,1005,9285,classic,1993
2,104,588,classic,1994
3,747,4915,classic,1994
4,70,439,classic,1994
5,577,3687,classic,1994
6,377,2429,classic,1994


## Client data

For the client dataset, we transformed the "birthnumber" into gender and birthdate, which we later used to calculate the client's age (as of the last transaction's date). We also dropped the client district, as we will be considering the account district only.

In [136]:
prepare_client <- function() {
  # Load dataset
  client_data <- read.csv("../data/client.csv", sep = ";")
  client_data <-
  replace(client_data, (client_data == "" | client_data == " "), NA)

  client_data <- transform(client_data,
    gender = ifelse(((birth_number %/% 100) %% 100) <= 12, 0, 1)
  )

  client_data <- transform(client_data, birthday = as.Date(
    paste(
      paste("19", birth_number %/% 10000, sep = ""),
      ifelse(((birth_number %/% 100) %% 100) <= 12,
        (birth_number %/% 100) %% 100,
        ((birth_number %/% 100) %% 100) - 50
      ),
      birth_number %% 100,
      sep = "-"
    ),
    format = "%Y-%m-%d"
  ))

  client_data <- subset(client_data, select = -c(birth_number, district_id))

  return(client_data)
}

In [137]:
client <- prepare_client()
head(client)

,client_id,gender,birthday
,<int>,<dbl>,<date>
1,1,1,1970-12-13
2,2,0,1945-02-04
3,3,1,1940-10-09
4,4,0,1956-12-01
5,5,1,1960-07-03
6,6,0,1919-09-22


## Disposition data

This dataset is pretty clean from the start so we can just load it.

In [138]:
prepare_disp <- function() {
  # Load dataset
  disp_data <- read.csv("../data/disp.csv", sep = ";")
  disp_data <-
    replace(disp_data, (disp_data == "" | disp_data == " "), NA)
}

In [139]:
disp <- prepare_disp()
head(disp)

,disp_id,client_id,account_id,type
,<int>,<int>,<int>,<chr>
1,1,1,1,OWNER
2,2,2,2,OWNER
3,3,3,2,DISPONENT
4,4,4,3,OWNER
5,5,5,3,DISPONENT
6,6,6,4,OWNER


## District data

The district dataset was one of the ones that needed more work done. For this one, we renamed some columns for concistency, filled the empty values with the column average where possible, and transformed the data from 1995 and 1996 into an average of both and an attribute that tells whether or not those numbers grew from an year to the other. We then dropped some of the columns that were no longer needed.

In [140]:
prepare_district <- function() {
  # Load dataset
  district_data <- read.csv("../data/district.csv",
    sep = ";", na.strings = c("NaN", "?"))
  district_data <-
    replace(district_data, (district_data == "" | district_data == " "), NA)

  # Rename code to district id to ease joins
  # Rename columns for consistency
  colnames(district_data)[colnames(district_data) == "code"] <- "district_id"
  colnames(district_data)[
    colnames(district_data) == "average.salary"
  ] <- "average_salary"
  colnames(district_data)[
    colnames(district_data) == "no..of.enterpreneurs.per.1000.inhabitants"
  ] <- "entrepreneur_rate"
  colnames(district_data)[
    colnames(district_data) == "no..of.inhabitants"
  ] <- "population"
  colnames(district_data)[
    colnames(district_data) == "ratio.of.urban.inhabitants"
  ] <- "urban_ratio"

  # Fix values that were "?" to be the column average
  district_data$unemploymant.rate..95[
    is.na(district_data$unemploymant.rate..95)
  ]  <- mean(as.numeric(district_data$unemploymant.rate..95), na.rm = TRUE)
  district_data$unemploymant.rate..96[
    is.na(district_data$unemploymant.rate..96)
  ] <- mean(as.numeric(district_data$unemploymant.rate..96), na.rm = TRUE)

  district_data$no..of.commited.crimes..95[
    is.na(district_data$no..of.commited.crimes..95)
  ] <- mean(as.numeric(district_data$no..of.commited.crimes..95), na.rm = TRUE)
  district_data$no..of.commited.crimes..96[
    is.na(district_data$no..of.commited.crimes..96)
  ] <- mean(as.numeric(district_data$no..of.commited.crimes..96), na.rm = TRUE)

  # Calculate average between 95 and 96
  district_data <- transform(district_data, unemployment_rate_avg =
    as.numeric(district_data$unemploymant.rate..95)
    + as.numeric(district_data$unemploymant.rate..96) / 2
  )
  district_data <- transform(district_data, crimes_rate_per_thousand =
    (as.numeric(district_data$unemploymant.rate..95)
    + as.numeric(district_data$unemploymant.rate..96) / 2)
    / as.numeric(district_data$population)
    * 1000
  )

  # Calculate whether or not the unemployment/crimes has been growing
  district_data <- transform(district_data, unemployment_growing =
    ifelse(
      as.numeric(district_data$unemploymant.rate..96) >
      as.numeric(district_data$unemploymant.rate..95),
      1,
      0
    )
  )
  district_data <- transform(district_data, crimes_growing =
    ifelse(
      as.numeric(district_data$no..of.commited.crimes..96) >
      as.numeric(district_data$no..of.commited.crimes..95),
      1,
      0
    )
  )


  district_data <- subset(district_data, select = -c(unemploymant.rate..95,
    unemploymant.rate..96, no..of.commited.crimes..95, no..of.commited.crimes..96,
    no..of.municipalities.with.inhabitants...499,
    no..of.municipalities.with.inhabitants.500.1999,
    no..of.municipalities.with.inhabitants.2000.9999,
    no..of.municipalities.with.inhabitants..10000,
    region,
    no..of.cities
  ))
  return(district_data)
}

In [141]:
district <- prepare_district()
head(district)

,district_id,name,population,urban_ratio,average_salary,entrepreneur_rate,unemployment_rate_avg,crimes_rate_per_thousand,unemployment_growing,crimes_growing
,<int>,<chr>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hl.m. Praha,1204953,100.0,12541,167,0.505,0.0004191035,1,1
2,2,Benesov,88884,46.7,8507,132,2.595,0.0291953557,1,1
3,3,Beroun,75232,41.7,8980,111,3.055,0.0406077201,1,0
4,4,Kladno,149893,67.4,9753,109,7.165,0.0478007645,1,1
5,5,Kolin,95616,51.4,9307,118,6.065,0.0634308066,1,1
6,6,Kutna Hora,77963,51.5,8546,126,4.960,0.0636199223,1,1


Like this, each district has a simpler representation, that is easier to use for comparisons between districts, as it uses statistics per thousand people instead of absolute counts.

## Loan data

For the loan data, we only needed to parse the date to YYYY-MM-DD
Since this table contains the target variable, we can alreay try to find some correlations.

In [142]:
prepare_loan <- function(train = TRUE) {
  loan_data <- read.csv(ifelse(train,
    "../data/loan_dev.csv", "../data/loan_comp.csv"),
    sep = ";"
  )
  loan_data <- replace(loan_data, (loan_data == "" | loan_data == " "), NA)
  loan_data <- transform(loan_data, date = as.Date(
    paste(
      paste("19", date %/% 10000, sep = ""),
      (date %/% 100) %% 100,
      date %% 100,
      sep = "-"
    ),
    format = "%Y-%m-%d"
  ))
  # payments * duration = amount, so don't need to keep them all
  loan_data <- subset(loan_data, select = -c(duration))

  return(loan_data)
}

In [143]:
loan <- prepare_loan()
head(loan)

,loan_id,account_id,date,amount,payments,status
,<int>,<int>,<date>,<int>,<int>,<int>
1,5314,1787,1993-07-05,96396,8033,-1
2,5316,1801,1993-07-11,165960,4610,1
3,6863,9188,1993-07-28,127080,2118,1
4,5325,1843,1993-08-03,105804,2939,1
5,7240,11013,1993-09-06,274740,4579,1
6,6687,8261,1993-09-13,87840,3660,1


# Transaction data

In [144]:
prepare_trans <- function(train = TRUE) {
  trans_data <- read.csv(ifelse(train,
    "../data/trans_dev.csv", "../data/trans_comp.csv"),
    sep = ";"
  )
  trans_data <- replace(trans_data, (trans_data == "" | trans_data == " "), NA)
  # Rename k_symbol column
  colnames(trans_data)[colnames(trans_data) == "k_symbol"] <- "category"
  trans_data$account <- replace(
    trans_data$account,
    (trans_data$account == 0), NA
  )

  # Make date more readable
  trans_data <- transform(trans_data, date = as.Date(
    paste(
      paste("19", date %/% 10000, sep = ""),
      (date %/% 100) %% 100,
      date %% 100,
      sep = "-"
    ),
    format = "%Y-%m-%d"
  ))

  # Make amount reflect if money entered or left the account
  trans_data <- transform(trans_data,
    amount = ifelse(type == "credit", amount, -1 * amount)
  )

  # Every entry where the operation is null, the k_symbol is defined as
  # "interest credited", so we can fill operation column with new type
  # - interest credited
  trans_data <- transform(trans_data,
    operation = ifelse(
      is.na(trans_data$operation) | trans_data$operation == "" |
        trans_data$operation == " ",
      "interest credited",
      operation
    )
  )
  trans_data$category[is.na(trans_data$category)] <- "other"

  # Drop type column: withdrawal in cash already in operation, withdrawal
  # vs credit already in amount sign
  trans_data <- subset(trans_data, select = -c(type))

  return(trans_data)
}

In [145]:
trans <- prepare_trans()
head(trans)

,trans_id,account_id,date,operation,amount,balance,category,bank,account
,<int>,<int>,<date>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>
1,1548749,5270,1993-01-13,credit in cash,800,800,other,NA,NA
2,1548750,5270,1993-01-14,collection from another bank,44749,45549,other,IJ,80269753
3,3393738,11265,1993-01-14,credit in cash,1000,1000,other,NA,NA
4,3122924,10364,1993-01-17,credit in cash,1100,1100,other,NA,NA
5,1121963,3834,1993-01-19,credit in cash,700,700,other,NA,NA
6,2809952,9307,1993-01-24,credit in cash,900,900,other,NA,NA


Now, we have to deal with the NA values. The only table that still has this problem after the initial cleaning is the transactions table. Since the percentage of null values in these cases is so high, there is nothing we can do to fill the values in an unbiased way. For that we have this helper function that removes every column where there are more than 70% null values.

In [146]:
remove_empty_cols <- function(data) {
  result <- data %>% select(where(~ mean(is.na(.)) < 0.7))
  return(result)
}

In [147]:
trans <- remove_empty_cols(trans)

In [148]:
colSums(is.na(trans))

trans_id account_id       date  operation     amount    balance   category 
         0          0          0          0          0          0          0

## Aggregate the transactions data

The transactions dataframe is the biggest one, and in order to relate it to the loan table, where our target variable is, we need to aggregate it by account_id.

In [149]:
aggregate_trans_data <- function(trans_data) {
  # Aggregate transactions data
  aggregated_trans <- trans_data %>%
    # Group by account
    group_by(account_id) %>%
    arrange(date, .by_group = TRUE) %>%
    # Add number of transactions per account
    mutate(trans_count = n()) %>%
    # Count credits/withdrawals
    mutate(credit_count = sum(amount >= 0)) %>%
    mutate(credit_ratio = mean(amount >= 0)) %>%
    mutate(withdrawal_count = sum(amount < 0)) %>%
    mutate(withdrawal_ratio = mean(amount < 0)) %>%
    # Amount stats
    mutate(smallest_transaction = amount[which.min(abs(amount))][1]) %>%
    mutate(biggest_transaction = amount[which.max(abs(amount))][1]) %>%
    mutate(transactions_net = sum(amount)) %>%
    # Balance stats
    mutate(balance_min = min(balance)) %>%
    mutate(balance_max = max(balance)) %>%
    mutate(current_balance = last(balance)) %>%
    mutate(times_negative_balance = sum(balance < 0)) %>%
    # Operation ratios
    mutate(credit_cash_ratio =
      mean(as.character(operation) == "credit in cash")) %>%
    mutate(collection_bank_ratio =
      mean(as.character(operation) == "collection from another bank")) %>%
    mutate(interest_ratio =
      mean(as.character(operation) == "interest credited")) %>%
    mutate(withdrawal_cash_ratio =
      mean(as.character(operation) == "withdrawal in cash")) %>%
    mutate(remittance_bank_ratio =
      mean(as.character(operation) == "remittance to another bank")) %>%
    mutate(withdrawal_card_ratio =
      mean(as.character(operation) == "credit card withdrawal")) %>%
    mutate(sanctions =
      sum(as.character(category) == "sanction interest if negative balance")) %>%
    rename(trans_date = date) %>%

    distinct()

  trans_agg <- subset(aggregated_trans, select =
    -c(trans_id, operation, amount, balance, category)
  )

  return(trans_agg)
}

In [150]:
trans <- aggregate_trans_data(trans)
head(trans)

account_id,trans_date,trans_count,credit_count,credit_ratio,withdrawal_count,withdrawal_ratio,smallest_transaction,biggest_transaction,transactions_net,⋯,balance_max,current_balance,times_negative_balance,credit_cash_ratio,collection_bank_ratio,interest_ratio,withdrawal_cash_ratio,remittance_bank_ratio,withdrawal_card_ratio,sanctions
<int>,<date>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1995-03-24,113,50,0.4424779,63,0.5575221,-14.6,12600,12659.9,⋯,30414.8,12674.5,0,0.07079646,0.1858407,0.1858407,0.4159292,0.1415929,0,0
1,1995-04-13,113,50,0.4424779,63,0.5575221,-14.6,12600,12659.9,⋯,30414.8,12674.5,0,0.07079646,0.1858407,0.1858407,0.4159292,0.1415929,0,0
1,1995-04-23,113,50,0.4424779,63,0.5575221,-14.6,12600,12659.9,⋯,30414.8,12674.5,0,0.07079646,0.1858407,0.1858407,0.4159292,0.1415929,0,0
1,1995-04-30,113,50,0.4424779,63,0.5575221,-14.6,12600,12659.9,⋯,30414.8,12674.5,0,0.07079646,0.1858407,0.1858407,0.4159292,0.1415929,0,0
1,1995-05-13,113,50,0.4424779,63,0.5575221,-14.6,12600,12659.9,⋯,30414.8,12674.5,0,0.07079646,0.1858407,0.1858407,0.4159292,0.1415929,0,0
1,1995-05-23,113,50,0.4424779,63,0.5575221,-14.6,12600,12659.9,⋯,30414.8,12674.5,0,0.07079646,0.1858407,0.1858407,0.4159292,0.1415929,0,0


With this, we created some new variables:

- **trans_count**: number of transactions for the account;
- **credit_count**: number of positive transactions for the account;
- **credit_ratio**: ratio of positive transactions for the account;
- **withdrawal_count**: number of negative transactions for the account;
- **withdrawal_ratio**: ratio of negative transactions for the account;
- **smallest_transaction**: smallest transaction for the account;
- **biggest_transaction**: biggest transaction for the account;
- **transactions_net**: difference of all money that entered and left the account;
- **balance_min**: lowest balance the account has had;
- **balance_max**: highest balance the account has had;
- **current_balance**: current balance;
- **times_negative_balance**: number of times the account's balance has been negative;
- **credit_cash_ratio**: ratio of "credit in cash" operations;
- **collection_bank_ratio**: ratio of "collection from another bank" operations;
- **interest_ratio**: ratio of "interest credited" operations;
- **withdrawal_cash_ratio**: ratio of "withdrawal in cash" operations;
- **remittance_bank_ratio**: ratio of "remittance to another bank" operations;
- **withdrawal_card_ratio**: ratio of "credit card withdrawal" operations;
- **sanctions**: number of "sanction interest if negative balance" operations;

And dropped the variables that were now redundant: trans_id, operation, amount, balance, date and category.

## Merging the data

Now that the data is ready, we will join every table and derive new attributes.

In [151]:
join_tables <- function(account_data, card_data, client_data,
                        disp_data, district_data, loan_data,
                        trans_data) {

  last_transaction <- max(trans_data$trans_date)
  trans_data <- trans_data %>%
    select(-trans_date)

  # Join tables and create more derived attributes
  data <- loan_data %>%
    rename(loan_date = date) %>%
    left_join(account_data, by = "account_id") %>%
    left_join(trans_data, by = "account_id") %>%
    mutate(transactions_net = transactions_net / age_days) %>%
    mutate(sanctions_rate = sanctions / age_days) %>%
    rename(daily_transactions_net = transactions_net) %>%
    left_join(disp_data, by = "account_id") %>%
    filter(type == "OWNER") %>%
    select(-age_days, -type, -sanctions) %>%
    left_join(card_data, "disp_id") %>%
    mutate(has_card = ifelse(!is.na(card_id), 1, 0)) %>%
    mutate(is_gold = ifelse((!is.na(type) & type == "gold"), 1, 0)) %>%
    select(-card_id, -type, -issued) %>%
    left_join(client_data, by = "client_id") %>%
    mutate(client_age = trunc(as.numeric(
      difftime(as.Date(last_transaction), birthday, units = "weeks")
    ) / 52.25)) %>%
    left_join(district_data, by = "district_id") %>%
    select(-c(name)) %>%
    mutate(can_afford_loan = ifelse(average_salary > payments, 1, 0)) %>%
    mutate(can_pay_until = current_balance / payments) %>%
    mutate(acc_age_when_loan = trunc(as.numeric(
      difftime(loan_date, acc_creation_date, units = "days")))
    ) %>%
    select(-c(acc_creation_date, account_id, district_id, date,
      disp_id, client_id, birthday
    )) %>%

    distinct()

  return(data)
}

In [152]:
data <- join_tables(account, card, client, disp, district, loan, trans)
head(data)

,loan_id,loan_date,amount,payments,status,frequency,trans_count,credit_count,credit_ratio,withdrawal_count,⋯,urban_ratio,average_salary,entrepreneur_rate,unemployment_rate_avg,crimes_rate_per_thousand,unemployment_growing,crimes_growing,can_afford_loan,can_pay_until,acc_age_when_loan
,<int>,<date>,<int>,<int>,<int>,<chr>,<int>,<int>,<dbl>,<int>,⋯,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5314,1993-07-05,96396,8033,-1,2,4,4,1.0000000,0,⋯,81.8,9650,100,5.215,0.0550035860,1,0,1,2.502179,105
2,5316,1993-07-11,165960,4610,1,1,37,17,0.4594595,20,⋯,73.5,8369,117,2.945,0.0261292355,1,0,1,11.325141,148
3,6863,1993-07-28,127080,2118,1,1,24,15,0.6250000,9,⋯,53.5,8390,132,3.725,0.0478072821,1,1,1,9.571671,170
4,5325,1993-08-03,105804,2939,1,1,25,13,0.5200000,12,⋯,58.0,8754,137,5.985,0.0554834523,1,1,1,11.668152,185
5,7240,1993-09-06,274740,4579,1,2,27,13,0.4814815,14,⋯,100.0,12541,167,0.505,0.0004191035,1,1,1,8.985128,204
6,6687,1993-09-13,87840,3660,1,1,17,14,0.8235294,3,⋯,41.3,8598,123,4.400,0.0816008605,1,1,1,11.980273,124


This has also allowed us to derive new attributes and improve old ones, such as:

- **transactions_net**: is now the net amount of money transacted per day;
- **sanctions_rate**: number of sanctions per day;
- **has_card**: whether or not there is a credit card associated with the account;
- **is_gold**: whether or not the account has a gold credit card;
- **client_age**: the age of the client as of the last transaction's date;
- **can_afford_loan**: whether or not the client's district has an average salary that can cover the loan's monthly payments;
- **can_pay_until**: how many months can the client's balance cover the loans payments for;
- **acc_age_when_loan**: how old the account was when the loan was made;

We also dropped some columns such as: age_days, type, sanctions, card_id, issued, acc_creation_date, account_id, district_id, date, disp_id, client_id.

We have developed a function `prepare_datasets(train)` that preforms all the previous operations allowing us to quickly get data ready for both traininig and testing.

In [155]:
prepare_datasets <- function(train = TRUE) {
  account_data <- prepare_account()
  client_data <- prepare_client()
  disp_data <- prepare_disp()
  district_data <- prepare_district()
  card_data <- prepare_card(train)
  loan_data <- prepare_loan(train)
  trans_data <- prepare_trans(train)

  trans_data <- remove_empty_cols(trans_data)
  trans_data <- aggregate_trans_data(trans_data)

  data <- join_tables(account_data, card_data, client_data,
                        disp_data, district_data, loan_data,
                        trans_data)

  if (!train) data <- data %>% mutate(status = c(NA))
  write.csv(data, ifelse(train, "../data/train.csv", "../data/test.csv"), row.names = FALSE)
}

In [156]:
prepare_datasets(TRUE) # Training dataset
prepare_datasets(FALSE) # Testing dataset